In [21]:
using Pkg
if !haskey(Pkg.installed(), "Images") || !haskey(Pkg.installed(), "ImageMagick") || !haskey(Pkg.installed(), "FileIO") || !haskey(Pkg.installed(), "Glob")
    println("Installing required packages...")
    Pkg.add.(["Images", "ImageMagick", "FileIO", "Glob"])
end



using LinearAlgebra
using GenericLinearAlgebra
using Images
using FileIO
using Glob
using ImageMagick  # pode ser necessário para JPEG


┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\Users\mathe\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Pkg.jl:744
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\Users\mathe\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Pkg.jl:744
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\Users\mathe\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Pkg.jl:744
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\Users\mathe\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Pkg.jl:744


##  Objetivo

Dado uma **matriz unitária ou ortogonal** $A \in \mathbb{R}^{(m+n) \times (p+q)}$, queremos fazer a **CS decomposition**:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S & 0 \\
S & C & 0 \\
0 & 0 & I
\end{bmatrix}
\begin{bmatrix}
V_1^\top & 0 \\
0 & V_2^\top
\end{bmatrix}
$$

Esse é o caso mais geral (com bloco identidade no final se sobrarem dimensões), mas vamos focar no **caso simples** para clareza:

### Suponha:

$$
A \in \mathbb{R}^{(m + n) \times r},\quad \text{com } m = n = r
\Rightarrow A \in \mathbb{R}^{2r \times r}
$$

E seja:

$$
A = \begin{bmatrix}
A_1 \\
A_2
\end{bmatrix},\quad A_1, A_2 \in \mathbb{R}^{r \times r}
$$




## Passo a passo da CSD via SVD


### - **Passo 1: Obter os subespaços**

Seja:

* $A_1 \in \mathbb{R}^{r \times r}$
* $A_2 \in \mathbb{R}^{r \times r}$

Você quer decompor $A$ entendendo a **relação geométrica entre os subespaços linha** de $A_1$ e $A_2$.



### - **Passo 2: Aplicar SVD em $A_1$**

Faça:

$$
A_1 = U_1 \Sigma_1 V^\top
$$

* $U_1 \in \mathbb{R}^{r \times r}$ (ortogonal)
* $\Sigma_1 = \text{diag}(\sigma_1^{(1)}, \dots, \sigma_r^{(1)})$
* $V \in \mathbb{R}^{r \times r}$ (ortogonal)


### - **Passo 3: Aplicar SVD em $A_2$** usando **mesmo $V$**

Agora, defina:

$$
\tilde{A}_2 = A_2 V
$$

E aplique SVD em $\tilde{A}_2$:

$$
\tilde{A}_2 = U_2 \Sigma_2
$$

Você agora tem:

* $A_1 = U_1 \Sigma_1 V^\top$
* $A_2 = U_2 \Sigma_2 V^\top$

Essas decomposições **compartilham a mesma base $V$** (isso é importante e possível porque $A$ é unitária).



### - **Passo 4: Obter os ângulos principais**

Agora, queremos achar os ângulos $\theta_i$ tais que:

$$
\cos(\theta_i) = \sigma_i^{(1)}, \quad \sin(\theta_i) = \sigma_i^{(2)}
$$

Como $A$ é unitária:

$$
A_1^\top A_1 + A_2^\top A_2 = I
\Rightarrow \Sigma_1^2 + \Sigma_2^2 = I
$$

Portanto, podemos definir:

$$
\theta_i = \arccos(\sigma_i^{(1)}) = \arcsin(\sigma_i^{(2)})
$$





### - **Passo 5: Construir os blocos da CSD**

* $C = \mathrm{diag}(\cos(\theta_1), \dots, \cos(\theta_r))$
* $S = \mathrm{diag}(\sin(\theta_1), \dots, \sin(\theta_r))$

A matriz do meio da CSD será:

$$
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
$$

E a CSD completa é:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
V^\top
$$



##  Recapitulando (Fluxo)

1. **Particione** $A$ em $A_1$ e $A_2$
2. **SVD** de $A_1 = U_1 \Sigma_1 V^\top$
3. Defina $\tilde{A}_2 = A_2 V$, depois faça $\tilde{A}_2 = U_2 \Sigma_2$
4. Calcule $\theta_i = \arccos(\sigma_i^{(1)})$
5. Monte $C, S$, e a decomposição completa



###  Observação importante

Esse processo depende de $A$ ser **unitária/ortogonal** — ou seja, $A^\top A = I$. Caso contrário, essa relação $\Sigma_1^2 + \Sigma_2^2 = I$ **não se sustentaria**, e os cossenos e senos não formariam uma matriz de rotação.



In [22]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # D1: m × d (m amostras, d features) - dataset 1
    # D2: n × d (n amostras, d features) - dataset 2

    # Passo 1: QR dos transpostos (colunas de D1 e D2 como vetores base)
    Q1, R1 = qr(D1')  
    Q2, R2 = qr(D2') 
    
    
    if size(R1, 1) == size(R1, 2)
        m= size(Q1, 1)
        n = size(R1, 2)
        R1_full = zeros(m, n)
        R1_full[1:n, :] = R1
    else
        R1_full = R1
    end
    if size(R2, 1) == size(R2, 2)
        m= size(Q2, 1)
        n = size(R2, 2)
        R2_full = zeros(m, n)
        R2_full[1:n, :] = R2      
    else
        R2_full = R2  
    end
    

    # Passo 2: Ortonormaliza a união dos subespaços de Q1 e Q2
    Q_all, _ = qr([Q1 Q2])  # Q_all ∈ ℝ^{d × r}, com r ≤ m + n

    # Passo 3: Projeta Q1 e Q2 no subespaço comum Q_all
    A1 = Q_all' * Q1  # ∈ ℝ^{r × m}
    A2 = Q_all' * Q2  # ∈ ℝ^{r × n}
    
    # Passo 4: Constrói a matriz A com blocos
    top = hcat(A1, zeros(size(A1, 1), size(A2, 2)))    # [A1  0]
    bottom = hcat(zeros(size(A2, 1), size(A1, 2)), A2)  # [0   A2]
    A = vcat(top, bottom)  # ∈ ℝ^{2r × (m + n)}

    return A, Q_all, R1_full, R2_full
end

build_csd_input_matrix (generic function with 1 method)

In [23]:
function rebuild_datasets_matrices(A, Q_all, R1, R2)
    # A: matriz de entrada
    # Q_all: base ortonormal comum
    # R1: matriz triangular superior de D1
    # R2: matriz triangular superior de D2

    r = size(Q_all, 2)  # dimensão do subespaço comum

    A1 = A[1:r, 1:size(R1, 1)] 
    D1_reconstructed = (Q_all * (A1' \ R1))'
   
    A2 = A[r+1:end, end-size(R2,1)+1:end] 
    D2_reconstructed = (Q_all * (A2' \ R2))'
    
    # D1_reconstructed: matriz reconstruída de D1
    # D2_reconstructed: matriz reconstruída de D2

    # Retorna as matrizes reconstruídas
    return D1_reconstructed, D2_reconstructed
    
end

rebuild_datasets_matrices (generic function with 1 method)

In [24]:
# Exemplo com dados aleatórios
m = 5  # amostras de D1
n = 7  # amostras de D2
d = 6  # número de features

D1 = randn(m, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

A, Q_all, R1, R2 = build_csd_input_matrix(D1, D2)

([1.0 -1.1102230246251565e-16 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … -0.19099734221678227 -0.7267037208312754; 0.0 0.0 … -0.6270100626694789 -0.07676959868876326], LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}([0.9999999999999999 -1.1102230246251565e-16 … 0.3471784511026196 -0.05939091769294497; -0.5886997453215905 1.0 … 0.06813636932443157 -0.03748939072276952; … ; 0.12235731176206453 0.26575198758088076 … -0.19099734221678227 -0.7267037208312754; 0.10095984524000944 0.129228559408898 … -0.6270100626694789 -0.07676959868876326], [1.330995000735619 1.2801181428214847 … -0.552965890998062 0.0; 5.0e-324 1.3423464281908373 … -1.0855196051990417 0.0; … ; 1.0e-323 1.264763357037e-311 … 1.8193668035037207 0.0; 3.0e-323 1.2647633570686e-311 … 1.14e-322 0.0]), [4.394276612692873 0.09493293943014769 … 0.08550792067234037 -1.3646694173393317; 0.0 2.8358965871693385 … 1.932039110018559 -1.0590843407290778; … ; 0.0 0.0 … 0.0 1.8498059026198155; 0.0 0.0 … 0.0 0.0], [-2.

In [25]:
# Exibir a matriz A
display(A)

12×12 Matrix{Float64}:
  1.0          -1.11022e-16  …   0.0         0.0         0.0
  0.0           1.0              0.0         0.0         0.0
  5.55112e-17  -1.11022e-16      0.0         0.0         0.0
  1.11022e-16  -6.15827e-17      0.0         0.0         0.0
 -1.38778e-16   1.11022e-16      0.0         0.0         0.0
  1.94289e-16  -2.77556e-17  …   0.0         0.0         0.0
  0.0           0.0             -0.728578    0.347178   -0.0593909
  0.0           0.0             -0.25649     0.0681364  -0.0374894
  0.0           0.0              0.520358    0.268788    0.480606
  0.0           0.0             -0.279951   -0.610701    0.47968
  0.0           0.0          …   0.232809   -0.190997   -0.726704
  0.0           0.0             -0.0064693  -0.62701    -0.0767696

In [26]:
if A'*A ≈ I
    println("A matriz A é ortonormal")
else
    println("A matriz A não é ortonormal")
    
end

A matriz A é ortonormal


In [27]:
D1_reconstructed, D2_reconstructed = rebuild_datasets_matrices(A, Q_all, R1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
    
end

As matrizes reconstruídas são equivalentes às originais!


In [49]:
function load_images_as_dataset(folder_path::String)
    # Lista todos os arquivos JPEG na pasta
    image_paths = Glob.glob("*.jpg", folder_path)  # ou "*.jpeg"
    num_images = length(image_paths)

    if num_images == 0
        error("Nenhuma imagem encontrada na pasta.")
    end
    
    # Carrega uma imagem para obter dimensões
    img_sample = load(image_paths[1])
    img_gray = Gray.(img_sample)  # Converte cada pixel RGB para Gray
    println(size(img_gray))
    h, w = size(img_gray)
    
    println("Dimensões da imagem: $h x $w")

    # Inicializa matriz dataset
    dataset = zeros(Float64, num_images, h * w)

    for (i, path) in enumerate(image_paths)
        img = load(path)
        img_gray = Gray.(img)  # Converte cada pixel RGB para Gray
        dataset[i, :] = vec(img_gray)  # achata e insere como linha
    end

    return dataset
end


load_images_as_dataset (generic function with 1 method)

In [50]:
#"D:\Docs matheus\UFRJ\7-Setimo Periodo\Alg. Lininear Aplicada\Trabalho final\CSD of subspaces\ace of spades"

folder = raw"D:\Docs matheus\UFRJ\7-Setimo Periodo\Alg. Lininear Aplicada\Trabalho final\CSD of subspaces\ace of spades"
X = load_images_as_dataset(folder)
size(X)  # ex: (100, 50176) se houver 100 imagens


(224, 224)
Dimensões da imagem: 224 x 224


(181, 50176)

In [51]:
X

181×50176 Matrix{Float64}:
 0.996078    1.0         1.0         …  0.988235   1.0        1.0
 0.996078    0.996078    0.996078       1.0        0.996078   1.0
 0.286275    0.172549    0.270588       0.223529   0.121569   0.117647
 0.843137    0.85098     0.854902       0.847059   0.847059   0.862745
 0.996078    1.0         0.988235       0.988235   0.996078   0.996078
 0.996078    1.0         0.996078    …  0.0588235  0.0588235  0.0705882
 0.968627    0.94902     0.937255       0.65098    0.6        0.560784
 0.00784314  0.00784314  0.00784314     0.780392   0.854902   0.894118
 0.403922    0.423529    0.415686       0.466667   0.470588   0.454902
 0.443137    0.458824    0.490196       0.243137   0.239216   0.215686
 ⋮                                   ⋱                        ⋮
 0.541176    0.545098    0.545098       0.490196   0.482353   0.47451
 0.576471    0.392157    0.180392       0.32549    0.278431   0.262745
 0.921569    0.94902     1.0            0.941176   0.898039   0.854